In [52]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib qt
import numpy as np
import cv2 as cv
import os
from typing import List
import matplotlib.pyplot as plt

In [53]:
def custom_pyramid(img, scales: List[float]) -> list:
    img_copy = img.copy()

    resized_imgs = list()

    # generamos nuestra "piramide" de tamaños arbitrarios
    for scale in scales:
        resized_imgs.append(cv.resize(img_copy, None, fx=scale, fy=scale))
        
    return resized_imgs

In [54]:
# Ruta de la carpeta con las imágenes
directorio = "./images"

# Lista para guardar las imágenes en gris
imagenes_gris = []

# Recorremos los archivos del directorio
for archivo in os.listdir(directorio):
    ruta_completa = os.path.join(directorio, archivo)

    # Verificamos que sea un archivo y termine en una extensión de imagen
    if os.path.isfile(ruta_completa) and archivo.lower().endswith(('.png', '.jpg')):
        img = cv.imread(ruta_completa, cv.IMREAD_GRAYSCALE)
        if img is not None:
            imagenes_gris.append(img)
        else:
            print(f"No se pudo leer la imagen: {archivo}")

print(f"Se leyeron {len(imagenes_gris)} imágenes en blanco y negro")

Se leyeron 7 imágenes en blanco y negro


In [55]:
# img_aux = []
# for img4 in imagenes_gris:
#     img_aux.extend(custom_pyramid(img4, np.linspace(0.2, 2, 10)))
# imagenes_gris = img_aux

In [56]:
template = cv.imread('./template/pattern.png',0)

In [57]:
template_list = custom_pyramid(template, np.linspace(0.2, 2, 10))

In [58]:
len(template_list)

10

In [59]:
def match_template(img3, template):
    img_copy1 = img3.copy()
    template_copy = template.copy()
    w, h = template.shape[::-1]

    # --- Desenfoque Gaussiano ---
    img_blur = cv.GaussianBlur(img_copy1, (5,5), 0)
    template_blur = cv.GaussianBlur(template_copy, (5,5), 0)

    # --- Bordes con Canny ---
    img_edges = cv.Canny(img_blur, 50, 150)
    template_edges = cv.Canny(template_blur, 50, 150)


    # print(f"Buscando plantilla de tamaño {template_edges.shape[0]}x{template_edges.shape[1]} en imagen de tamaño {img_edges.shape[0]}x{img_edges.shape[1]}")
    res = cv.matchTemplate(img_edges, template_edges, cv.TM_CCOEFF_NORMED)
    # Fijamos un umbral de "coincidencia"

    threshold = 0.2
    flag = False
    loc = np.where( res >= threshold)
    for pt in zip(*loc[::-1]):
        cv.rectangle(img_copy1, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
        flag = True
    
    if flag:
        return img_copy1
    else:
        return None

In [60]:
def match_template2(img, template):
    # --- Preprocesar template ---
    template = template.copy()
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    template_blur = cv.GaussianBlur(template_gray, (5,5), 0)
    template_edges = cv.Canny(template_blur, 40, 160)

    h, w = template_edges.shape
    best_val = -1
    best_loc = None
    best_scale = None
    best_result = None

    # --- Recorrer pirámide de escalas ---
    for scale in np.linspace(0.25, 3, 40):
        resized = cv.resize(img, None, fx=scale, fy=scale)
        if resized.shape[0] < h or resized.shape[1] < w:
            continue  # saltar si el template no entra en la imagen

        # Preprocesar imagen escalada
        img_gray = cv.cvtColor(resized, cv.COLOR_BGR2GRAY)
        img_blur = cv.GaussianBlur(img_gray, (5,5), 0)
        img_edges = cv.Canny(img_blur, 40, 160)

        # Matching
        result = cv.matchTemplate(img_edges, template_edges, cv.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        if max_val > best_val:
            best_val = max_val
            best_loc = max_loc
            best_scale = scale
            best_result = resized.copy()

    # --- Verificar antes de dibujar ---
    if best_loc is not None:
        top_left = best_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        cv.rectangle(best_result, top_left, bottom_right, (255, 0, 0), 2)
        print(f"Mejor coincidencia: {best_val:.4f} en escala {best_scale:.2f}")
        # cv.imshow("Mejor resultado", best_result)
        # cv.waitKey(0)
        # cv.destroyAllWindows()
        return best_result
    else:
        print("No se encontró coincidencia.")
        return None

In [61]:
def search_template(img1, template_list):
    img_copy5 = img1.copy()

    result = []
    for i, template in enumerate(template_list):
        try:
            img9 = match_template2(img_copy5, template)
            if img9 is not None:
                result.append(img9)
            # cv.imshow(f'Template Level {i}', img_copy)
            # cv.waitKey(0)
        except Exception as e:
            # print(f"Error processing template level {i}: {e}")
            continue
    return result

In [62]:
def run_all_img(img_list, template_list):
    result = []
    for img2 in img_list:
        result.extend(search_template(img2, template_list))
    return result

In [63]:
r = run_all_img(imagenes_gris, template_list)

In [64]:
len(r)

0

In [65]:
cv.imshow(f'Template Level', r[0])

IndexError: list index out of range